<span style="color:darkorange; font-size:60px">Learn Jupyter with Bloomberg</span></br>
<span style="color:lightblue; font-size:30px">The Jupyter widgets ecosystem - introduction</span>

## A common analyst workflow

In [ ]:
# Importing relevant packages.
import requests # Accessing APIs.
import pandas as pd # Processing data.
import numpy as np # Processing data.
from datetime import timezone, datetime # Time data manipulation.
import matplotlib.pyplot as plt # Used for plots.
from utils.plot_utils import set_style # Helper function for nicer looking plots.
%matplotlib inline

In [ ]:
# Making plots look nicer on dark backgrounds.
set_style()

In [ ]:
API_TOKEN = '<YOUR_TOKEN_GOES_HERE>' # Register for a token on finnhub.io if you want to retrieve data with the API.

In [ ]:
# Unix date format (number of seconds since 01/01/1970)
start_date = datetime(2010, 10, 19).replace(tzinfo=timezone.utc).timestamp()
end_date = datetime(2020, 10, 2).replace(tzinfo=timezone.utc).timestamp()

In [ ]:
# Here's what it looks like.
print(start_date)

In [ ]:
# API call used to create currencies.csv

# currencies = requests.get('https://finnhub.io/api/v1/forex/symbol?exchange=oanda&token=<YOUR_FINNHUB_TOKEN_HERE>')
# pd.DataFrame(currencies.json()).to_csv('currencies.csv')

currencies = pd.read_csv('data/currencies.csv')

In [ ]:
# Examining the available currencies.
# currencies['displaySymbol'].tolist()

In [ ]:
# Use pandas masking to get the corresponding row with the data we are looking for.
currencies[currencies['displaySymbol'] == 'XAU/AUD']

In [ ]:
# Index into the column we are interested in retrieving data for.
currencies[currencies['displaySymbol'] == 'XAU/AUD']['symbol']

In [ ]:
# Using string templates in python to our advantage.
request_string = 'https://finnhub.io/api/v1/forex/candle?symbol={symbol}&resolution=D&from={start_date}&to={end_date}&token={token}'.format(symbol="OANDA:XAU_AUD", token=API_TOKEN, start_date=start_date, end_date=end_date)
request_string

In [ ]:
# Using the request_string to make an API call - not used in this webinar!
# xau_aud = requests.get(request_string)
# pd.DataFrame(xau_aud.json()).to_csv('xau_aud.csv')

In [ ]:
# Close, high, low, open, status, timestamp, volume.
xau_aud = pd.read_csv('data/XAU_AUD.csv')
xau_aud

In [ ]:
# Notice the timestamp column. We need a more useful format.
xau_aud['t'] = xau_aud['t'].apply(datetime.fromtimestamp)

In [ ]:
# Setting the 't' column as our new index.
xau_aud.set_index('t', inplace=True)
xau_aud.head(3)

In [ ]:
# Removing the "Unnamed: 0" column from our data frame.
xau_aud = xau_aud[xau_aud.columns[1:]]
xau_aud.head(3)

In [ ]:
# Friendlier name for the x-axis.
xau_aud.index.rename('Date', inplace=True)
xau_aud.head(3)

In [ ]:
# Quick visualization of the closing price.
plt.show(xau_aud['c'].plot(title='XAU/AUD Price', figsize=(16,9)));

### This workflow is great for a one-off. But it can be challenging if needs to be reproduced for each currency:
* Getting the right symbol from the dataframe and typing it each time.
* Generating a request string each time the start/end times or currency pair changes.
* Manipulating the data after it has been retrieved so that we have a readable timestamp
* Plotting the end result

<span style="color: orange">Repetitive code is not a recommended practice in software engineering and data science. If all we want is a nice interface to select the currency pair, start date, end date, and plot the result, surely there must be a better way.</span>

<span style="color: lightblue; font-size:30px"> Interactive Widgets </span>

### ipywidgets - interactive widgets library for Jupyter Notebooks
* [ipywidgets documentation](https://ipywidgets.readthedocs.io/en/latest/index.html)
* [widgets list](https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html)
* [github repository](https://github.com/jupyter-widgets/ipywidgets)

### What is a widget?
```
"an application, or a component of an interface, that enables a user to perform a function or access a service" (Oxford Languages)
```
<span style="color: orange">In our context, widgets are small GUI components with a well-defined and specific purpose, used together to compose an application.</span>

Examples:
* Button
* Dropdown list
* Slider
* Layout box
* Graph
* much, much more...

In [ ]:
# Button example.
import ipywidgets as ipw

In [ ]:
# Output widget (optional).
output_widget = ipw.Output()

In [ ]:
# Creating a button
my_button = ipw.Button(description="Click me", button_style="success")
my_button

In [ ]:
# Do something when the the button is clicked.
@output_widget.capture()
def do_something(value):
    string_to_print = "'value' is the widget instance being triggered: {}".format(value)
    print(string_to_print)

In [ ]:
# Assigning a callback function which is triggered each time the button is clicked.
my_button.on_click(do_something)

In [ ]:
# Displaying the Output widget which will print stdout.
output_widget

In [ ]:
# Clearing the output widget area
output_widget.clear_output()

## Congratulations! You just created your first widget-powered application, and how easy was it?!

<span style="color: limegreen">So how does it work, exactly?</span>

<img src='https://ipywidgets.readthedocs.io/en/latest/_images/WidgetModelView.png'>

### Let's take a quick look at the "Widget_List" notebook to explore other widget types

# Can this solve our problem of not a having dynamic data retrieval and plotting solution for our currencies? Yes it can.

In [ ]:
# Extratcing the description from our 'currencies' data frame.
symbol_list = currencies['description'].tolist()

In [ ]:
# Uncomment this to see the list.
# symbol_list

In [ ]:
# Dropdown widget for the currency list.
drop_down = ipw.Dropdown(options=symbol_list)
drop_down

In [ ]:
# What is the value currently displayed by the drop down widget? Try selecting different values and re-execute this cell.
print(drop_down.value)

In [ ]:
# ..and the corresponding symbol
currencies[currencies['description'] == drop_down.value]['displaySymbol']

In [ ]:
currencies[currencies['description'] == symbol_list[0]]['symbol'].values[0]

In [ ]:
# For your reference - code used to retrieve the currency .csv files in the data directory.

# from time import sleep
# import os

# i = 0
# while i < len(symbol_list):
#     if (i % 29 == 0):
#         print('A short pause..') # Finnhub doesn't allow more than 30 requests per second.
#         sleep(2)
        
#     ccy_symbol = currencies[currencies['description'] == symbol_list[i]]['symbol'].values[0]
#     file_name = currencies[currencies['description'] == symbol_list[i]]['displaySymbol'].values[0]
#     request_string = 'https://finnhub.io/api/v1/forex/candle?symbol={symbol}&resolution=D&from={start_date}&to={end_date}&token={token}'.format(symbol=ccy_symbol, token=API_TOKEN, start_date=start_date, end_date=end_date)
#     file_exists = os.path.exists(os.getcwd()+ "/data/" + file_name.replace("/","_") + '.csv')
#     if not file_exists:
#         print(file_name.replace("/","_") + '.csv' + " missing!")
#         pd.DataFrame(requests.get(request_string).json()).to_csv("data/" + file_name.replace("/","_") + '.csv')
#     i += 1

In [ ]:
# Constructing our control bar.
start_date_selector = ipw.DatePicker(description="Start date", value=datetime(2018,1,1))
end_date_selector = ipw.DatePicker(description="End date", value=datetime(2020,8,5))
get_data_button = ipw.Button(description="Get data!", button_style='success')

control_bar = ipw.HBox([drop_down, start_date_selector, end_date_selector, get_data_button])
control_bar

In [ ]:
# Adding our output area.
output_area = ipw.Output()

In [ ]:
# Combining the two.
# app_container = ipw.VBox([control_bar])
app_container = ipw.VBox([control_bar, output_area])

In [ ]:
# Uncomment to see what the container looks like (will not look different from control_bar without any output!)
# app_container

### The 'observe' pattern
<span style="color:orange">We previously saw how we can listen to a 'click' even on a button and when that event happens, a specific function gets called. The observe pattern serves the same purpose - to trigger a function when an event happens, only this time by 'observing' a specific value of a widget and then triggering a function when that specific observed trait has changed.</span>

Let's observe a change in the __value__ trait `drop_down` widget and trigger a print function when that value changes. 

In [ ]:
@output_area.capture()
def drop_down_callback(value):
    output_area.clear_output()
    print("Previously selected currency: {}. New selected currency: {}".format(value['old'], value['new'])) # The value object passed into the callback is a regular python dictionary with ipywidgets properties.

In [ ]:
# Adding the event listener which will observe the 'value' trait of the dropdown and trigger drop_down_callback() each time it changes.
drop_down.observe(drop_down_callback, 'value')

### Try it selecting a different currency from the drop down menu!

In [ ]:
# Clearing the output area.
output_area.clear_output()

<span style="color:lightblue; font-size:30px">Finishing our application - putting everything together</span>

In [ ]:
# Get data callback.
@output_area.capture(clear_output=True) # This allows the output widget to cleare the content each time this function is called - keeps things neat!
def get_data_callback(value):
    # Checking for the right name format.
    if '/' in drop_down.value:
        file_name = currencies[currencies['description'] == drop_down.value]['displaySymbol'].values[0].replace("/","_")+'.csv'
    else:
        file_name = drop_down.value + '.csv'

    # Loading the correct file into a dataframe.
    currency_data = pd.read_csv('data/'+file_name)
    
    # Converting the 't' column data type from Unix timestamp to python datetime (which is easier to work with).
    currency_data['t'] = currency_data['t'].apply(datetime.fromtimestamp)
    
     # Setting the 't' as an index column (notice we're doing this in-place).
    currency_data.set_index('t', inplace=True)
    
    # Removing the "Unnamed: 0" and volume columns from our data frame.
    currency_data = currency_data[currency_data.columns[1:-1]]
    
    # Renaming the x-axis to something friendlier.
    currency_data.index.rename('Date', inplace=True)
    
    # Friendlier names for the OHLC columns
    currency_data.rename({'o':'Open','h':'High','l':'Low','c':'Close'}, axis=1, inplace=True)
    
    # Selecting the currency data based on the date ranges in our date selectors.
    selected_data = currency_data.loc[start_date_selector.value:end_date_selector.value] # The .loc method has a [index, column] format and allows for an index range such as .loc[start:end, :]. The colon means "all columns" in this case.
    
    # Rendering a matplotlib figure with a title corres
    plt.show(selected_data.plot(title=drop_down.value, figsize=(16,9)))

In [ ]:
# Adding an event listener for the button click.
get_data_button.on_click(get_data_callback)

In [ ]:
app_container